In [1]:
import pandas as pd
import random
import warnings
import os
import re
warnings.filterwarnings("ignore")

from dotenv import load_dotenv

load_dotenv()
token = os.getenv("HUGGING_FACE_API_KEY")

file_path = '/home/halil/Desktop/MeETU/Data/FAQs_eng.xlsx'
excel_data = pd.read_excel(file_path)

In [2]:
from huggingface_hub import InferenceClient
import json

repo_id = "mistralai/Mistral-7B-Instruct-v0.3"

llm_client = InferenceClient(
    model=repo_id,
    timeout=120,
    token=token
)
def call_llm(inference_client: InferenceClient, prompt: str):
    response = inference_client.post(
        json={
            "inputs": prompt,
            "parameters": {"max_new_tokens": 1000, "temparature": 1},
            "task": "text-generation",
        },
    )
    return json.loads(response.decode())[0]["generated_text"]


In [3]:
def generate_synthetic_question(answer):
    prompt = (
        f"You are generating a question that a prospective student at Middle East Technical University (METU) "
        f"might ask based on the following information. Write a concise, small and clear question with simple words that captures the key details:\n\n"
        f"Information: '{answer}'"
    )
    synthetic_question = call_llm(llm_client, prompt)

    return synthetic_question.strip()


In [4]:
faq_data = excel_data
faq_data['synthetic_question'] = faq_data['answer'].apply(generate_synthetic_question)

In [18]:
print(faq_data.synthetic_question[47])

You are generating a question that a prospective student at Middle East Technical University (METU) might ask based on the following information. Write a concise, small and clear question with simple words that captures the key details:

Information: '

 METU VPN Service Use Policy
Aim of METU VPN Service
The aim of the METU VPN service is, to enable our staff and students to access the network services which are only permitted to METU campus IP block (144.122.0.0/16), even if they are out of the campus.
 Rules for Using METU VPN Service
1. The METU VPN Service, which is provided to METU staff and students, should be used in accordance with the METU Information Technology Resources Use Policy which is given on the link
2. The state allocates and grants to the University financial resources in order to accomplish computer networking technology investments that will primarily aid the instructional, academic, research and administrative objectives of the University. The personal use of th

In [20]:
for i in faq_data.synthetic_question:
    _text = i.split("'\n\n")
    if len(_text) == 3:
        print(_text[-1].strip())

"How do I connect to eduroam Wi-Fi at METU using my Windows 10 device without installing any program?"
Question: "What special characters should I avoid when creating a password for the meturoam wireless network at METU, and what should I do if I'm having trouble connecting with Android or iOS devices?"
Question: "Can I connect my Android device running on Android 6 or earlier to the METU wireless network, and if so, what should I do if I face connection problems?"
"How can I access the Library Electronic Resources from outside METU Campus after the Webcache Service ended on August 5th, 2014?"
"What is the website link to learn about METU's Wired Network Departments/Units?"
Question:

"Why can't I connect to my computer at METU from outside the campus using remote desktop, and how can I change the port number to establish the connection?"
Question: "How can I set a new password for my METU account?"
"Is the fingerprint of the Radius Server for the meturoam network at METU the same as t